### CONNECTING Python to MySQL..week3

### TASK:  In this lab, you will be working with the Sakila database on movie rentals. Specifically, you will be practicing how to do basic SQL queries using Python. By connecting Python to SQL, you can leverage the power of both languages to efficiently manipulate and analyze large datasets. 

In [1]:
pip install pymysql pandas sqlalchemy

  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/e5/30/20467e39523d0cfc2b6227902d3687a16364307260c75e6a1cb4422b0c62/PyMySQL-1.1.0-py3-none-any.whl.metadata
  Using cached PyMySQL-1.1.0-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.0-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from sqlalchemy import create_engine

# Replace these variables with your actual database connection details
username = 'root'
password = '13113629'
database_name = 'sakila'
host = 'localhost'  # or your database host
port = '3306'  # default MySQL port

# Create an engine to connect to the MySQL database
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}')


NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:mysql.pymysql

### retrieving the rental data

In [6]:
import pandas as pd

def rentals_month(engine, month, year):
    query = f"""
    SELECT * FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    return pd.read_sql_query(query, engine)


### counting the rentals by customers / month

In [7]:
def rental_count_month(df, month, year):
    # Group by customer_id and count rentals
    result = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return result


In [ ]:
### comparing the rentals on a monthly basis

In [8]:
def compare_rentals(df1, df2):
    # Ensure the customer_id columns are of the same type
    df1['customer_id'] = df1['customer_id'].astype(str)
    df2['customer_id'] = df2['customer_id'].astype(str)
    
    # Merge the two dataframes on customer_id
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    
    # Calculate the difference in rentals between the two periods
    column_names = df1.columns[1] + '_' + df2.columns[1]  # Assumes the second column contains the rental count
    merged_df['difference'] = merged_df.iloc[:, 1] - merged_df.iloc[:, 2]
    
    return merged_df
